# Blue Brain BioExplorer
![](../../../bioexplorer_banner.png)

In [ ]:
from bioexplorer import BioExplorer, MovieMaker
import os

url = 'localhost:5000'
resource_folder = os.path.abspath('../../../tests/test_files')
    
be = BioExplorer(url)
core = be.core_api()
mm = MovieMaker(be)
version = be.version()
print('Version ' + version)

sequence_name = 'high_glucose'

## Data loading

In [ ]:
'''Reset current scene'''
be.reset_scene()

In [ ]:
'''Accelerate loading by not showing models as they are loaded'''
status = be.set_general_settings(
    logging_enabled=True,
    model_visibility_on_creation=False)

In [ ]:
cache_folder = os.path.join(resource_folder, 'caches', version)

In [ ]:
cache_filename = cache_folder + sequence_name + '_scenario.bioexplorer'
print('Loading ' + cache_filename)
dummy = core.add_model(cache_filename)

In [ ]:
nb_models = len(core.scene.models)
if nb_models != 180:
    print('ERROR: Not all models have been loaded!!! Expected 180, found ' + str(nb_models))
else:
    print('All good!')

In [ ]:
'''Now we can show the models'''
status = be.set_models_visibility(True)

## Rendering settings

In [ ]:
status = be.set_rendering_quality(be.RENDERING_QUALITY_HIGH)

## Image settings

In [ ]:
image_size = [3840, 2160]
image_samples_per_pixel = 64

output_folder = os.path.join('/tmp', 'frames', version, sequence_name)

command_line='mkdir -p ' + output_folder
os.system(command_line)
command_line='ls ' + output_folder
if os.system(command_line) != 0:
    print('ERROR: Failed to create output folder')

## Snapshot

In [ ]:
def set_all_materials_to_grey():
    for model in core.scene.models:
        model_id = model['id']
        model_name = model['name']

        material_ids = be.get_material_ids(model_id)['ids']
        nb_materials = len(material_ids)
        palette = list()
        for p in range(nb_materials):
            palette.append([0.25,0.25,0.25])
        be.set_materials(
            model_ids=[model_id], material_ids=material_ids, 
            diffuse_colors=palette, specular_colors=palette)

In [ ]:
def apply_glucose_colors():
    import seaborn as sns
    specular_exponent = 50
    for model in core.scene.models:
        model_id = model['id']
        model_name = model['name']

        material_ids = be.get_material_ids(model_id)['ids']
        nb_materials = len(material_ids)

        if be.NAME_GLUCOSE in model_name:
            palette = sns.color_palette('Blues', nb_materials)
            be.set_materials_from_palette(
                specular_exponent=specular_exponent,
                model_ids=[model_id], material_ids=material_ids, palette=palette)

### Frame 1100 - Surfactant head

In [ ]:
mm.set_camera(
    direction=[0.27479800000000004, -0.048306, -0.960206],
    origin=[38.749, 35.228, 5.535],
    up=[0.012606000000000001, 0.99883, -0.046638]
)

In [ ]:
set_all_materials_to_grey()
apply_glucose_colors()

In [ ]:
mm.create_snapshot(
    renderer='advanced',
    size=image_size, samples_per_pixel=image_samples_per_pixel,
    path=output_folder, base_name='01100')